In [5]:
import pandas as pd
import numpy as np

import folium 
from folium import plugins

In [7]:
from ChernobylUtility import UsefulFunctions as uf

In [8]:
air = pd.read_csv("air_modified.csv")

In [31]:
def fill_a_point(index, df):
    """ Fill up a point for the folium map using data for the ith index, and an air data frame. 
    
    index, df -> Dictionary containing results. 
    """
    coordinates = [ df.loc[ index, 'latitude'], df.loc[ index, 'longitude'] ]
    
    begin_wrap = '<h1> '
    end_wrap = ' </h1>'
    city = df.loc[index, 'city']
    country_code = df.loc[index, 'country_code']
    popup = begin_wrap + city + ', ' + country_code + end_wrap
    
    time = df.loc[index, 'Date']
    
    result = { 'coordinates': coordinates, 'popup': popup, 'time': time }
    return result

In [84]:
def populate_points(df):
    """ Fill up the points list with points following the fill a point format. 
    
    df -> list of points 
    """
    points = []
    for index in range(len(df)):
        single_point = fill_a_point(index, df)
        points.append(single_point)
    return points

In [85]:
def populate_point_features(dict_points):
    """ Get a list of dictionaries containing separate points from the data. 
    
    list with dictionaries of points -> output a list containing features for the points
    """
    features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': point['coordinates'],
        },
        'properties': {
            'time': point['time'],
            'icon': 'circle',
            'popup': point['popup'],
            'iconstyle': {
                'fillColor': 'green',
                'fillOpacity':0.6,
                'stroke':'false',
                'radius':5
            },
            'style': {'weight': 0}
        }
    } for point in dict_points
    ]
    return features

In [86]:
points = populate_points(air)

In [87]:
features = populate_point_features(points)

In [88]:
# Notice latitude is the first coordinate
CHERNOBYL_COORDS = [51.386998452, 30.092666296]
ZOOM_START = 3
folium_map = folium.Map( location = CHERNOBYL_COORDS , zoom_start = ZOOM_START)

In [89]:
plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1D',
    add_last_point=False,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY-MM-DD',
    time_slider_drag_update=True
).add_to(folium_map)

In [90]:
folium_map